In [12]:
# basic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

#tensorflow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

#keras
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
# Load the dataset
base_dir = "../dataset"
df_data_original = pd.read_csv(f"{base_dir}/dataset.csv")

# make sure there is no NaN value
assert df_data_original.isnull().values.any() == False

In [3]:
df_data_complete = df_data_original.drop(labels=["skn", "year", "month", "lat", "lon", "Lon_DD_updated"], axis=1)

In [4]:
df_data_complete.head()

,data_in,Lat_DD,Lon_DD,air,air.1,hgt,hgt.1,omega,pottmp,pottmp.1,pr_wtr,shum,shum.1,shum.2,shum.3,shum.4,shum.5,skt,slp
0,3.200000,18.916176,-155.674994,295.39603,31.299995,5799.5483,121.48387,0.023998,-26.433594,-3.926514,29.034512,2.592493,-25.859348,0.589191,7.106412,2.945999,9.869999,23.385218,1014.0849
1,5.950000,19.108660,-155.825545,295.39603,31.299995,5799.5483,121.48387,0.023998,-26.433594,-3.926514,29.034512,2.592493,-25.859348,0.589191,7.106412,2.945999,9.869999,23.385218,1014.0849
2,11.500000,19.164740,-155.682280,295.39603,31.299995,5799.5483,121.48387,0.023998,-26.433594,-3.926514,29.034512,2.592493,-25.859348,0.589191,7.106412,2.945999,9.869999,23.385218,1014.0849
3,5.515941,19.160603,-155.822488,295.39603,31.299995,5799.5483,121.48387,0.023998,-26.433594,-3.926514,29.034512,2.592493,-25.859348,0.589191,7.106412,2.945999,9.869999,23.385218,1014.0849
4,4.310617,19.225323,-155.778876,295.39603,31.299995,5799.5483,121.48387,0.023998,-26.433594,-3.926514,29.034512,2.592493,-25.859348,0.589191,7.106412,2.945999,9.869999,23.385218,1014.0849


In [5]:
Y = df_data_complete["data_in"].to_numpy()
X = df_data_complete.drop(["data_in"], axis=1).to_numpy()

In [59]:
X.shape

(865561, 18)

In [60]:
X.reshape(-1, 18, 1).shape

(865561, 18, 1)

In [6]:
scaler = StandardScaler()
Xtemp, Xtest, Ytemp, Ytest = train_test_split(X, Y, test_size=0.2)
Xtrain, Xvalid, Ytrain, Yvalid = train_test_split(Xtemp, Ytemp, test_size=0.25)

In [8]:
Xtrain = scaler.fit_transform(Xtrain)
Xvalid = scaler.transform(Xvalid)
Xtest = scaler.transform(Xtest)

In [9]:
# rmodel = LinearRegression()
# rmodel.fit(Xtrain.reshape(-1,18), Ytrain)
# yhat = rmodel.predict(Xtest.reshape(-1,18))
# mse(Ytest, yhat)

122549218.80962351

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience = 3, mode='min')
# construct a model
model = models.Sequential()
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, kernel_initializer='normal'))

# model.add(layers.Conv1D(64, 8, activation = 'relu', input_shape=(19, 1)))
# model.add(layers.MaxPool1D(5))
# model.add(layers.Conv1D(32, 4, activation = 'relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation = 'relu'))
#model.add(layers.Dense(32, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(1, activation='softmax'))
epochs = 20
model.compile(
    optimizer = 'adam',
    loss = 'mean_squared_error'
)
history = model.fit(
    Xtrain, Ytrain, 
    epochs=epochs, 
    validation_data = (Xvalid, Yvalid),
    callbacks=[callback],
    batch_size=256
)

2022-02-04 02:05:42.697015: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-04 02:05:42.698666: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-04 02:05:42.745666: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:1b:00.0 name: NVIDIA GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2022-02-04 02:05:42.745717: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-02-04 02:05:42.748325: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-02-04 02:05:42.748405: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.

Epoch 1/20


2022-02-04 02:05:44.124311: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


2029/2029 [==============================] - 5s 2ms/step - loss: 41.1165 - val_loss: 29.4711
Epoch 2/20
2029/2029 [==============================] - 5s 2ms/step - loss: 29.0200 - val_loss: 27.4713
Epoch 3/20
2029/2029 [==============================] - 5s 2ms/step - loss: 27.7570 - val_loss: 26.5134
Epoch 4/20
2029/2029 [==============================] - 4s 2ms/step - loss: 26.7489 - val_loss: 25.9782
Epoch 5/20
2029/2029 [==============================] - 4s 2ms/step - loss: 26.1706 - val_loss: 25.7026
Epoch 6/20
2029/2029 [==============================] - 4s 2ms/step - loss: 25.7342 - val_loss: 25.5987
Epoch 7/20
2029/2029 [==============================] - 4s 2ms/step - loss: 25.8308 - val_loss: 25.5403
Epoch 8/20
2029/2029 [==============================] - 4s 2ms/step - loss: 25.5719 - val_loss: 25.4186
Epoch 9/20
2029/2029 [==============================] - 4s 2ms/step - loss: 24.9967 - val_loss: 25.2794
Epoch 10/20
2029/2029 [==============================] - 4s 2ms/step - loss

In [49]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(19, input_dim=19, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Baseline: nan (nan) MSE


/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 10.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/yusukemh/.conda/envs/climate/lib/python3.9/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 157, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "/tmp/ipykernel_234848/1957097577.py", line 3, in bas

In [13]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1
